In [1]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
import scipy.stats as stats
from matplotlib import gridspec
import aplpy as apl #The Astronomy Plotting Library for python
import astropy
from astropy.stats import sigma_clip
from astropy.modeling import functional_models, models, fitting
import astropy.units as U
from astropy.coordinates import ICRS, Galactic, FK4, FK5, Angle, Latitude, Longitude
import astropy.constants as C
from astropy import wcs
import astropy.io.fits as fits
from astropy.io import ascii
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.stats import signal_to_noise_oir_ccd as SNRas
#import spectral_cube as SC
#from spectral_cube import SpectralCube as sc
from astropy.wcs import WCS
matplotlib.use('Agg')
import matplotlib.cm as cm
#import astrometry as ast
import pyfits
import requests
import json
import os
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
import numpy.ma as ma
from astropy.table import Table, Column
import pyextract
import sewpy
import pandas as pd
from astropy import units as u

/home/lynge/anaconda2/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/lynge/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/lynge/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/lynge/anaconda2

In [2]:
sta1_RA_WORLD = SkyCoord('13:06:23.368 -40:24:47.36', unit=(u.hourangle, u.deg)).ra.degree #TYC 7777-7-1
sta1_DEC_WORLD = SkyCoord('13:06:23.368 -40:24:47.36', unit=(u.hourangle, u.deg)).dec.degree

sta2_RA_WORLD = SkyCoord('13:06:20.816 -40:24:00.86', unit=(u.hourangle, u.deg)).ra.degree #RAVE J130620.8-402401
sta2_DEC_WORLD = SkyCoord('13:06:20.816 -40:24:00.86', unit=(u.hourangle, u.deg)).dec.degree

sta3_RA_WORLD = SkyCoord('13:06:29 -40:20:56', unit=(u.hourangle, u.deg)).ra.degree #UCAC2 14614825
sta3_DEC_WORLD = SkyCoord('13:06:31.543 -40:29:53.57', unit=(u.hourangle, u.deg)).dec.degree

sky_RA_WORLD = SkyCoord('13:06:26 -40:25:33', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
sky_DEC_WORLD = SkyCoord('13:06:26 -40:25:33', unit=(u.hourangle, u.deg)).dec.degree


k_g = 0.16
k_r = 0.08
k_i = 0.02
k_z = 0.01

mag_refsta1_g = 11.861
mag_refsta1_r = 11.562
mag_refsta1_i = 11.460
mag_refsta1_z = 11.430

mag_refsta2_g = 10.166
mag_refsta2_r = 10.229
mag_refsta2_i = 10.347
mag_refsta2_z = 10.456

mag_refsta3_g = 10.166
mag_refsta3_r = 10.229
mag_refsta3_i = 10.347
mag_refsta3_z = 10.456

refsta1_RA = SkyCoord('01:54:38 00:43:01', unit=(u.hourangle, u.deg)).ra.degree #SA093_317
refsta1_DEC = SkyCoord('01:54:38 00:43:01', unit=(u.hourangle, u.deg)).dec.degree 

refsta2_RA = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).ra.degree #PG1323_086_10_07_17
refsta2_DEC = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).dec.degree 

refsta3_RA = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).ra.degree #PG1323_086_10_07_17
refsta3_DEC = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).dec.degree 


In [3]:
QUASAR1_g = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-ESO323/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-ESO323/g')]

QUASAR2_g = [os.path.join('/media/lynge/LACIE/Calibration/28-05-17-ESO323/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/28-05-17-ESO323/g')]

QUASAR3_g = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-ESO323/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-ESO323/g')]

refsta1_g = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/g')]

refsta2_g = [os.path.join('/media/lynge/LACIE/Calibration/28-05-17-SA100_241/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/28-05-17-SA100_241/g')]

refsta3_g = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/g')]

In [4]:
QUASAR1_r = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-ESO323/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-ESO323/r')]

QUASAR2_r = [os.path.join('/media/lynge/LACIE/Calibration/28-05-17-ESO323/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/28-05-17-ESO323/r')]

QUASAR3_r = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-ESO323/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-ESO323/r')]

refsta1_r = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/r')]

refsta2_r = [os.path.join('/media/lynge/LACIE/Calibration/28-05-17-SA100_241/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/28-05-17-SA100_241/r')]

refsta3_r = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/r')]

In [5]:
QUASAR1_i = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-ESO323/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-ESO323/i')]

QUASAR2_i = [os.path.join('/media/lynge/LACIE/Calibration/28-05-17-ESO323/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/28-05-17-ESO323/i')]

QUASAR3_i = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-ESO323/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-ESO323/i')]

refsta1_i = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/i')]

refsta2_i = [os.path.join('/media/lynge/LACIE/Calibration/28-05-17-SA100_241/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/28-05-17-SA100_241/i')]

refsta3_i = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/i')]

In [6]:
QUASAR1_z = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-ESO323/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-ESO323/z')]

QUASAR2_z = [os.path.join('/media/lynge/LACIE/Calibration/28-05-17-ESO323/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/28-05-17-ESO323/z')]

QUASAR3_z = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-ESO323/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-ESO323/z')]

refsta1_z = [os.path.join('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/04-01-17-SA093_317/z')]

refsta2_z = [os.path.join('/media/lynge/LACIE/Calibration/28-05-17-SA100_241/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/28-05-17-SA100_241/z')]

refsta3_z = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/z')]

In [7]:
def dist(mid,end):
    return np.sqrt((mid[0] - end[0])**2 + (mid[1] - end[1])**2)

In [8]:
def flux_cal(A_mag,B_mag,B_flux):
    '''Flux of A found through magnitude and flux of B'''
    return B_flux*10**((B_mag-A_mag)/2.5)

In [9]:
def Basic(quasar,header):
    '''Provides a basic list of neccesities from the fits header file for use in the later \
    flux determination, it does not serve an individual purpose beyound that'''
    AGNdata = quasar #np.rot90(np.rot90(np.fliplr(quasar)))
    #header = data[0].header
    #print header
    RAstd = header['CRVAL1']
    DECstd = header['CRVAL2']
    #print RAstd, DECstd
    pixspa = header['PIXSCALE']/(3600.)
    #print pixspa
    RAstdpix = header['CRPIX1']
    DECstdpix = header['CRPIX2']
    exptime = 10 # header['EXPTIME']
    return header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime

In [10]:
def curve(quasar,header,w,center,apparature):
    '''Determines the FLUX of the stellar object given the the numpy array of the image, \
    the header of the fits file, the astropy coordinate representation, \
    the object position and the apparature of interest'''
    header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime = Basic(quasar,header)
    #AGNdata = np.swapaxes(AGNdata,0,1)
    y, x = np.ogrid[0:header['NAXIS1'],0:header['NAXIS1']]
    #w = np.swapaxes(w,0,1)
    #F_AGN = np.zeros((512,512,3)) #((int(2*apparature/pixspa)+1,int(2*apparature/pixspa)+1,3))
    #print center
    x1 = w.all_world2pix(center[0],center[1],0)
    x1,y1 = (x1[0]),(x1[1])
    try:
        mask = ((y-y1)**2 + (x-x1)**2) > (apparature/float(pixspa))**2
        AGNdata[mask] = float(0)
        mask = AGNdata == 0.
        rows = np.flatnonzero((~mask).sum(axis=1))
        cols = np.flatnonzero((~mask).sum(axis=0))
        AGNdata = AGNdata[rows.min():rows.max()+1, cols.min():cols.max()+1]
        #print apparature/float(pixspa)
    except:
        AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    #if np.sum(AGNdata) < 500:
    #    AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    return AGNdata/float(exptime), x1, y1


In [11]:
def comp1(qua,sta):
    '''Code to determine the relation between the AGN and the Stellar flux as determined by either me \
    or by the SExtractor program respectively'''
    comp = qua/sta
    return comp

In [12]:
def SNR(data1,data2,header,ap1,ap2):
    npix1 = np.pi*ap1**2/((header['PIXSCALE']/3600.)**2)
    npix2 = np.pi*ap2**2/((header['PIXSCALE']/3600.)**2)
    #data2_1 = ((data2-data1)/npix2)*npix1
    data1_2 = data1/10.
    data2_2 = data2/10.
    return SNRas(10.,data1_2,abs(data2_2),0,header['RON'],npix1,gain=header['GAIN'])

In [13]:
def SXtable(quas,aperture):
    header = pyfits.getheader(quas)
    phot_app = aperture*3600/header['PIXSCALE']
    np.set_printoptions(threshold=np.inf)
    sew = sewpy.SEW(params=['XMODEL_WORLD', 'YMODEL_WORLD', 'X2MODEL_WORLD', 'Y2MODEL_WORLD', 
                            'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_RADIUS(1)', 'FLUX_APER','ELLIPTICITY'],
                    config={"DETECT_THRESH":5, 'DETECT_MINAREA':20, 'DETECT_MAXAREA':30000, 
                            "PHOT_FLUXFRAC":"0.5", 'PHOT_APERTURES':2*phot_app, 
                            'PIXEL_SCALE':header['PIXSCALE']}
                    , sexpath='/usr/bin/sextractor')
    out = sew(quas)
    return out

In [14]:
def flux(quas,aperture,main_RA_WORLD,main_DEC_WORLD):
    header = pyfits.getheader(quas)
    RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[0]
    DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[1]
    output = float('nan')
    if 15 < RAmain < header['NAXIS1'] - 15 and 15 < DECmain < header['NAXIS2'] - 15: 
        out = SXtable(quas,aperture)
        #print out['table']['XMODEL_IMAGE']
        #print RAmain
        RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[0]
        DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[1]
        #print out['table']['XMODEL_IMAGE'], out['table']['YMODEL_IMAGE']
        #print RAmain, DECmain

        for j in range(len(out['table']['XMODEL_IMAGE'])):
            #output['time'][i] = header['MJD-OBS']
            if RAmain - 20. < out['table']['XMODEL_IMAGE'][j] <  RAmain + 20. and \
            DECmain - 20. < out['table']['YMODEL_IMAGE'][j] < DECmain + 20.:
                #print 'YES'
                output = out['table']['FLUX_APER'][j]
                break
    else:
        output = float('nan')
        #print 'Done'
    return output


In [15]:
def findmag(start,compare1,RA_star,DEC_star,RA_com,DEC_com,mag_com,AIR_EXTINCTION):
    print mag_com
    aperture1 = 0.0012
    #201.5441,-8.8601
    com_flu = []
    com_AIRMASS = []
    com_EXPTIME = []
    star_flu = []
    star_AIRMASS = []
    star_EXPTIME = []
    for i in range(len(compare1)):
        print 'Frame nr. ', i, compare1[i]
        header = pyfits.getheader(compare1[i])
        a1 = flux(compare1[i],2*aperture1,RA_com,DEC_com)/float(header['EXPTIME'])
        print header['EXPTIME']
        if a1 > 0:
            #print a1
            com_flu.append(a1), com_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), com_EXPTIME.append(header['EXPTIME'])
        else:
            print a1
            com_flu.append(float('nan')), com_AIRMASS.append(float('nan')), com_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    for i in range(len(start)):
        print 'Frame nr. ', i, start[i]
        header = pyfits.getheader(start[i])
        a1 = flux(start[i],2*aperture1,RA_star,DEC_star)/float(header['EXPTIME'])
        print header['EXPTIME']
        if a1 > 0:
            #print a1
            star_flu.append(a1), star_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), star_EXPTIME.append(header['EXPTIME'])
        else:
            star_flu.append(float('nan')), star_AIRMASS.append(float('nan')), star_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    com_flu = np.array(com_flu)
    print com_flu
    com_AIRMASS = np.array(com_AIRMASS)
    com_EXPTIME = np.array(com_EXPTIME)
    star_flu = np.array(star_flu)
    print star_flu
    star_AIRMASS = np.array(star_AIRMASS)
    star_EXPTIME = np.array(star_EXPTIME)
    com_flu = com_flu#/com_EXPTIME
    star_flu = star_flu#/star_EXPTIME
    com_flux = np.nanmean(com_flu)
    com_AIRMASS_num = np.nanmean(com_AIRMASS)
    star_AIRMASS_num = np.nanmean(star_AIRMASS)
    print star_AIRMASS_num
    mag1 = (star_AIRMASS_num-com_AIRMASS_num) + mag_com - 2.5*np.log10(star_flu/com_flux)
    mag = mag1
    print (star_AIRMASS_num-com_AIRMASS_num)
    print mag_com, com_flux, np.average(com_flu)
    print mag
    return mag

In [16]:
com1_1 = findmag(QUASAR1_g,refsta1_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com2_1 = findmag(QUASAR1_g,refsta1_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com3_1 = findmag(QUASAR1_g,refsta1_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)

com1_2 = findmag(QUASAR2_g,refsta2_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)
com2_2 = findmag(QUASAR2_g,refsta2_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)
com3_2 = findmag(QUASAR2_g,refsta2_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)

com1_3 = findmag(QUASAR3_g,refsta3_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_g,k_g)
com2_3 = findmag(QUASAR3_g,refsta3_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_g,k_g)
com3_3 = findmag(QUASAR3_g,refsta3_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_g,k_g)

11.861
Frame nr.  0 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/g/IMG2017002URh009.fits


/home/lynge/anaconda2/lib/python2.7/site-packages/pyfits/hdu/image.py:515: VerifyWarning: Invalid 'BLANK' keyword in header.  The 'BLANK' keyword is only applicable to integer data, and will be ignored in this HDU.
  warnings.warn(msg, VerifyWarning)
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
No handlers could be found for logger "sewpy.sewpy"


40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/g/IMG2017002URh010.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/g/IMG2017002URh013.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/g/IMG2017002URh014.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/g/IMG2017002URs011.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/g/IMG2017002URs012.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/g/IMG2017002URs015.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/g/IMG2017002URs016.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/04-01-17-ESO323/g/IMG2017002URs036.fits
45.0
Frame nr.  1 /media/lynge/LACIE/Calibration/04-01-17-ESO323/g/IMG2017002URs037.fits
45.0
Frame nr.  2 /media/lynge/LACIE/Calibration/04-01-17-ESO323/g/IMG2017002URs038.fits
45.0
[12308.0125 12311.1775 12401.6225 12323.015  12339.47   12423.015
 12441.415  12411.

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: Mean of empty slice


40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/g/IMG2017146URh002.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/g/IMG2017146URh005.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/g/IMG2017146URh006.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/g/IMG2017146URs003.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/g/IMG2017146URs004.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/g/IMG2017146URs007.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/g/IMG2017146URs008.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/28-05-17-ESO323/g/IMG2017146URs039.fits
45.0
Frame nr.  1 /media/lynge/LACIE/Calibration/28-05-17-ESO323/g/IMG2017146URs040.fits
45.0
Frame nr.  2 /media/lynge/LACIE/Calibration/28-05-17-ESO323/g/IMG2017146URs041.fits
45.0
[61032.825 61480.4   61364.575 61466.25  61701.75  61824.65  61701.1
 61595.45 ]
[13

In [17]:
print 'g-filter'
print 'Star 1'
print com1_1,com1_2,com1_3
print np.nanmean((com1_1,com1_2,com1_3)), np.nanstd((com1_1,com1_2,com1_3))

print 'Star 2'
print com2_1,com2_2,com2_3
print np.nanmean((com2_1,com2_2,com2_3)), np.nanstd((com2_1,com2_2,com2_3))

print 'Star 3'
print com3_1,com3_2,com3_3
print np.nanmean((com3_1,com3_2,com3_3)), np.nanstd((com3_1,com3_2,com3_3))

g-filter
Star 1
[12.59167729 12.57397124 12.5438845 ] [11.76451141 11.74684124 11.73877898] [11.96930884 11.95640083 11.9552572 ]
12.09340350286222 0.34792125974926114
Star 2
[13.83911217 13.82107194 13.79228811] [12.90018619 12.88608611 12.8776627 ] [13.19664501 13.19667934 13.19286448]
13.300288450128129 0.38685121249220117
Star 3
[nan nan nan] [nan nan nan] [nan nan nan]
nan nan


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':
/home/lynge/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [18]:
com1_1 = findmag(QUASAR1_r,refsta1_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com2_1 = findmag(QUASAR1_r,refsta1_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com3_1 = findmag(QUASAR1_r,refsta1_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)

com1_2 = findmag(QUASAR2_r,refsta2_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)
com2_2 = findmag(QUASAR2_r,refsta2_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)
com3_2 = findmag(QUASAR2_r,refsta2_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)

com1_3 = findmag(QUASAR3_r,refsta3_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_r,k_r)
com2_3 = findmag(QUASAR3_r,refsta3_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_r,k_r)
com3_3 = findmag(QUASAR3_r,refsta3_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_r,k_r)

11.562
Frame nr.  0 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/r/IMG2017002BRh009.fits
40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/r/IMG2017002BRh010.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/r/IMG2017002BRh013.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/r/IMG2017002BRh014.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/r/IMG2017002BRs011.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/r/IMG2017002BRs012.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/r/IMG2017002BRs015.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/r/IMG2017002BRs016.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/04-01-17-ESO323/r/IMG2017002BRs036.fits
45.0
Frame nr.  1 /media/lynge/LACIE/Calibration/04-01-17-ESO323/r/IMG2017002BRs037.fits
45.0
Frame nr.  2 /media/lynge/LACIE/Calibration/04-01-17-ESO323/r/IMG2017002BRs038.

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: Mean of empty slice


40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/r/IMG2017146BRh002.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/r/IMG2017146BRh005.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/r/IMG2017146BRh006.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/r/IMG2017146BRs003.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/r/IMG2017146BRs004.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/r/IMG2017146BRs007.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/r/IMG2017146BRs008.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/28-05-17-ESO323/r/IMG2017146BRs039.fits
45.0
Frame nr.  1 /media/lynge/LACIE/Calibration/28-05-17-ESO323/r/IMG2017146BRs040.fits
45.0
Frame nr.  2 /media/lynge/LACIE/Calibration/28-05-17-ESO323/r/IMG2017146BRs041.fits
45.0
[63153.05 63375.8  63274.8  63495.85 63696.85 63762.15 63596.35 63726.25]
[19394.317

In [19]:
print 'r-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

r-filter
Star 1
[11.97937987 11.96315866 11.94771585] [11.49913203 11.48464329 11.47880418]
11.725472314647634 0.23819800796456453
Star 2
[12.33167816 12.32267404 12.29776672] [11.69955822 11.68409943 11.68017169]
12.002658043467983 0.3149338927107012
Star 3
[nan nan nan] [nan nan nan]
nan nan


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


In [20]:
com1_1 = findmag(QUASAR1_i,refsta1_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com2_1 = findmag(QUASAR1_i,refsta1_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com3_1 = findmag(QUASAR1_i,refsta1_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)

com1_2 = findmag(QUASAR2_i,refsta2_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)
com2_2 = findmag(QUASAR2_i,refsta2_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)
com3_2 = findmag(QUASAR2_i,refsta2_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)

com1_3 = findmag(QUASAR3_i,refsta3_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_i,k_i)
com2_3 = findmag(QUASAR3_i,refsta3_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_i,k_i)
com3_3 = findmag(QUASAR3_i,refsta3_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_i,k_i)

11.46
Frame nr.  0 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/i/IMG2017002ULh009.fits
40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/i/IMG2017002ULh010.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/i/IMG2017002ULh013.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/i/IMG2017002ULh014.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/i/IMG2017002ULs011.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/i/IMG2017002ULs012.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/i/IMG2017002ULs015.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/i/IMG2017002ULs016.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/04-01-17-ESO323/i/IMG2017002ULs036.fits
45.0
Frame nr.  1 /media/lynge/LACIE/Calibration/04-01-17-ESO323/i/IMG2017002ULs037.fits
45.0
Frame nr.  2 /media/lynge/LACIE/Calibration/04-01-17-ESO323/i/IMG2017002ULs038.f

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: Mean of empty slice


40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/i/IMG2017146ULh002.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/i/IMG2017146ULh005.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/i/IMG2017146ULh006.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/i/IMG2017146ULs003.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/i/IMG2017146ULs004.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/i/IMG2017146ULs007.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/i/IMG2017146ULs008.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/28-05-17-ESO323/i/IMG2017146ULs039.fits
45.0
Frame nr.  1 /media/lynge/LACIE/Calibration/28-05-17-ESO323/i/IMG2017146ULs040.fits
45.0
Frame nr.  2 /media/lynge/LACIE/Calibration/28-05-17-ESO323/i/IMG2017146ULs041.fits
45.0
[32710.925 32719.5   32791.2   32871.725 33015.495 33048.595 33006.34
 32999.415]
[1

In [21]:
print 'i-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

i-filter
Star 1
[11.60027755 11.5789188  11.57560309] [11.43355867 11.41804392 11.41318144]
11.503263910870706 0.0822641915218133
Star 2
[10.85124725 10.84748876 10.82953312] [10.57625937 10.55722068 10.54990898]
10.701943027147303 0.1411912607110283
Star 3
[nan nan nan] [nan nan nan]
nan nan


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


In [22]:
com1_1 = findmag(QUASAR1_z,refsta1_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com2_1 = findmag(QUASAR1_z,refsta1_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com3_1 = findmag(QUASAR1_z,refsta1_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)

com1_2 = findmag(QUASAR2_z,refsta2_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)
com2_2 = findmag(QUASAR2_z,refsta2_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)
com3_2 = findmag(QUASAR2_z,refsta2_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)

com1_3 = findmag(QUASAR3_z,refsta3_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_z,k_z)
com2_3 = findmag(QUASAR3_z,refsta3_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_z,k_z)
com3_3 = findmag(QUASAR3_z,refsta3_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta3_RA,refsta3_DEC,mag_refsta3_z,k_z)

11.43
Frame nr.  0 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/z/IMG2017002BLh009.fits
40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/z/IMG2017002BLh010.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/z/IMG2017002BLh013.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/z/IMG2017002BLh014.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/z/IMG2017002BLs011.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/z/IMG2017002BLs012.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/z/IMG2017002BLs015.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/04-01-17-SA093_317/z/IMG2017002BLs016.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/04-01-17-ESO323/z/IMG2017002BLs036.fits
45.0
Frame nr.  1 /media/lynge/LACIE/Calibration/04-01-17-ESO323/z/IMG2017002BLs037.fits
45.0
Frame nr.  2 /media/lynge/LACIE/Calibration/04-01-17-ESO323/z/IMG2017002BLs038.f

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: Mean of empty slice


40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/z/IMG2017146BLh002.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/z/IMG2017146BLh005.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/z/IMG2017146BLh006.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/z/IMG2017146BLs003.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/z/IMG2017146BLs004.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/z/IMG2017146BLs007.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/28-05-17-SA100_241/z/IMG2017146BLs008.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/28-05-17-ESO323/z/IMG2017146BLs039.fits
45.0
Frame nr.  1 /media/lynge/LACIE/Calibration/28-05-17-ESO323/z/IMG2017146BLs040.fits
45.0
Frame nr.  2 /media/lynge/LACIE/Calibration/28-05-17-ESO323/z/IMG2017146BLs041.fits
45.0
[5991.4   6030.77  5989.72  6013.355 6010.905 6037.36  6026.715 6047.24 ]
[2545.5866

In [23]:
print 'z-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

z-filter
Star 1
[11.5258843  11.49465228 11.51742895] [11.38801521 11.38185769 11.37613231]
11.447328457127432 0.06607825931173189
Star 2
[10.20007482 10.2045724  10.19476637] [9.93736542 9.92387676 9.92084517]
10.063583489467774 0.13634509127703517
Star 3
[nan nan nan] [nan nan nan]
nan nan


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


In [24]:
HD101274_1 = findmag(NGC3783_r,P_r,sta1_RA_WORLD,sta1_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_22_1 = findmag(NGC3783_r,P_r,sta2_RA_WORLD,sta2_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_529_1 = findmag(NGC3783_r,P_r,sta3_RA_WORLD,sta3_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)




NameError: name 'NGC3783_r' is not defined

In [ ]:
print 'r-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

In [ ]:
HD101274_1 = findmag(NGC3783_i,P_i,sta1_RA_WORLD,sta1_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_22_1 = findmag(NGC3783_i,P_i,sta2_RA_WORLD,sta2_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_529_1 = findmag(NGC3783_i,P_i,sta3_RA_WORLD,sta3_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)




In [ ]:
print 'i-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

In [ ]:
sta2_RA_WORLD = 174.82083333
sta2_DEC_WORLD = -37.76194444

HD101274_1 = findmag(NGC3783_z,P_z,sta1_RA_WORLD,sta1_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_22_1 = findmag(NGC3783_z,P_z,sta2_RA_WORLD,sta2_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_529_1 = findmag(NGC3783_z,P_z,sta3_RA_WORLD,sta3_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)




In [ ]:
print 'z-filter'
print HD101274_1
print np.nanmean(HD101274_1,), np.nanstd(HD101274_1)

print Tyc_22_1
print np.nanmean(Tyc_22_1), np.nanstd(Tyc_22_1)

print Tyc_529_1
print np.nanmean(Tyc_529_1), np.nanstd(Tyc_529_1)